# Hand Landmarks
![Hand Land Marks](https://mediapipe.dev/images/mobile/hand_landmarks.png)

In [ ]:
import cv2
import numpy as np
import os
import pickle

# Define paths
DATA_DIR = 'data'  # Adjust if your data folder has a different name
output_file = '5- Hand_landmarks_Data.pickle'

# Initialize lists to store data
data = []
labels = []

# Define a function to detect hands using skin color and contours
def detect_hand(image):
    # Convert to HSV color space for skin detection
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    # Define skin color range in HSV
    lower_skin = np.array([0, 20, 70], dtype=np.uint8)
    upper_skin = np.array([20, 255, 255], dtype=np.uint8)
    mask = cv2.inRange(hsv, lower_skin, upper_skin)

    # Apply morphological operations to reduce noise
    kernel = np.ones((5, 5), np.uint8)
    mask = cv2.dilate(mask, kernel, iterations=2)
    mask = cv2.erode(mask, kernel, iterations=2)

    # Find contours
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return None

    # Get the largest contour (assumed to be the hand)
    max_contour = max(contours, key=cv2.contourArea)
    if cv2.contourArea(max_contour) < 500:  # Ignore small contours
        return None

    # Get bounding box of the hand
    x, y, w, h = cv2.boundingRect(max_contour)
    # Normalize coordinates (relative to image size)
    img_height, img_width = image.shape[:2]
    normalized_features = [x/img_width, y/img_height, (x+w)/img_width, (y+h)/img_height]
    return normalized_features

# Loop through each class folder in the data directory
for label in os.listdir(DATA_DIR):
    label_dir = os.path.join(DATA_DIR, label)
    if not os.path.isdir(label_dir):
        continue

    print(f'Processing label: {label}')
    for img_name in os.listdir(label_dir):
        img_path = os.path.join(label_dir, img_name)
        img = cv2.imread(img_path)
        if img is None:
            continue

        # Detect hand and extract features
        features = detect_hand(img)
        if features is None:
            continue

        # Append features and label to lists
        data.append(features)
        labels.append(label)

# Save the data to a pickle file
data_dict = {'data': data, 'labels': labels}
with open(output_file, 'wb') as f:
    pickle.dump(data_dict, f)

print(f'Saved {len(data)} samples to {output_file}')

c:\Users\Mohamed Bayomey\AppData\Local\Programs\Python\Python311\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
